In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np
import re
import tldextract
import html
import scrape_schema_recipe

In [2]:
data = pd.read_csv('/Users/vincentsalamand/Downloads/recipes_export.csv')
curated_data = data[data.origin != 'mama']

FileNotFoundError: [Errno 2] File b'/Users/vincentsalamand/Downloads/recipes_export.csv' does not exist: b'/Users/vincentsalamand/Downloads/recipes_export.csv'

In [27]:
websites = []

for link in curated_data.link:
    ext = tldextract.extract(link)
    websites.append('.'.join(ext))

websites = set(websites)
len(websites)

64

In [542]:
valid_websites = [
 'amandebasilic.com',
 'cahierdegourmandises.fr',
 'clemfoodie.com',
 'cuisine-addict.com',
 'cuisinemoiunmouton.com',
 'deliacious.com',
 'emiliemurmure.com',
 'fourneauxetfourchettes.fr',
 'freethepickle.fr',
 'mahealthytendency.com',
 'mesbrouillonsdecuisine.fr',
 'tangerinezest.com',
 'www.cuisine-et-mets.com',
 'www.cuisine-etudiant.fr',
 'www.cuisineactuelle.fr',
 'www.cuisineaz.com',
 'www.cuisineculinaire.com',
 'www.delizioso.fr',
 'www.elle.fr',
 'www.marciatack.fr',
 'www.marieclaire.fr',
 'www.papillesetpupilles.fr',
 'www.recettesdici.com',
 'www.regal.fr',
 'www.ricardocuisine.com',
 'www.safrangourmand.fr',
 'www.tastemade.fr',
 'www.thegoodmiam.com',
 'www.undejeunerdesoleil.com',
 'www.unjourunerecette.fr'
]

In [2]:
# 3rd party parser that scrapes schema_org 
url = 'https://clemfoodie.com/2020/05/04/risotto-aux-asperges-vertespetis-pois-parmesan/'
recipe_list = scrape_schema_recipe.scrape_url(url, python_objects=True)
pd.DataFrame(recipe_list)


,@context,@id,@type,author,cookTime,datePublished,description,image,isPartOf,keywords,mainEntityOfPage,name,prepTime,recipeCategory,recipeIngredient,recipeInstructions,recipeYield,totalTime,url
0,http://schema.org/,https://clemfoodie.com/2020/05/04/risotto-aux-...,Recipe,"{'@type': 'Person', 'name': 'Clemfoodie'}",00:20:00,2020-05-04 19:10:29+00:00,,[https://clemfoodie.com/wp-content/uploads/202...,{'@id': 'https://clemfoodie.com/2020/05/04/ris...,"asperges, oeuf poché, parmesan, petits pois, r...",https://clemfoodie.com/2020/05/04/risotto-aux-...,"Risotto aux asperges vertes, petits pois, parm...",00:05:00,"[Déjeuner, Dîner]","[1 c. à soupe d'huile d'olive, 2 gousses d'ai...","[{'@type': 'HowToStep', 'text': 'Emincez l'ail...","[2, 2 personnes]",00:25:00,https://clemfoodie.com/2020/05/04/risotto-aux-...


In [755]:
driver = webdriver.Chrome("/Applications/chromedriver")


In [767]:
category_urls = []
for number in range(41):
    base_url = "https://www.papillesetpupilles.fr/category/recettes-salees/page/"
    category_urls.append(base_url + str(number))

category_urls = category_urls[1:]

In [768]:
len(category_urls)

40

In [777]:
#driver.get(category_urls[0])
#soup = BeautifulSoup(driver.page_source, 'lxml')
soup.find_all('div', class_="image")[0].find('a')['href']


'https://www.papillesetpupilles.fr/2020/02/cuisses-de-poulet-a-la-moutarde-et-au-thym.html/'

In [778]:
# scrape website recipe urls
recipe_urls = []
#website = "https://cuisine-addict.com/index-recettes/"


def scraper(website):
    driver.get(website)
    sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    get_recipe_urls(soup)
    #try:
   #     next_page = soup.find('a', class_="next")['href']
    #    print(next_page)
     #   scraper(next_page)
    #except:
        #print("end of category")
     #   pass


def get_recipe_urls(soup):
    [recipe_urls.append(article.find('a')['href']) for article in soup.find_all('div', class_="image")]
#    [recipe_urls.append(article.find('a')['href']) for article in soup.find('div', class_="blog-feed").find_all('div', class_="entry-thumb")]

#scraper(website)
for website in category_urls:
    scraper(website)

    

In [779]:
print(len(recipe_urls))
content = []

for recipe_url in recipe_urls:
    try:
        recipe = scrape_schema_recipe.scrape_url(recipe_url, python_objects=True)
        content.append(recipe)
        print(recipe_url)
    except:
        print("exception")
        print(recipe_url)
        pass

recipes = [item for sublist in content for item in sublist]
recipe_data = pd.DataFrame(recipes)


2375
https://www.papillesetpupilles.fr/2020/02/cuisses-de-poulet-a-la-moutarde-et-au-thym.html/
https://www.papillesetpupilles.fr/2020/02/pizza-blanche-aux-champignons.html/
https://www.papillesetpupilles.fr/2020/02/carottes-aux-epices-douces.html/
https://www.papillesetpupilles.fr/2020/02/aiguillettes-de-canard-au-miel-vinaigre-balsamique-et-sesame.html/
https://www.papillesetpupilles.fr/2020/02/polenta-cremeuse.html/
https://www.papillesetpupilles.fr/2020/02/aiguillettes-de-poulet-a-la-moutarde-au-miel-et-au-curry.html/
https://www.papillesetpupilles.fr/2020/02/chouchous-christophines-au-jambon-et-bechamel.html/
https://www.papillesetpupilles.fr/2020/02/gratin-de-poireaux-saumon-et-parmesan.html/
https://www.papillesetpupilles.fr/2020/02/veloute-de-champignons.html/
https://www.papillesetpupilles.fr/2020/02/ceviche-de-cabillaud-aux-pamplemousses-roses.html/
https://www.papillesetpupilles.fr/2020/02/les-achards-de-papaye-verte-de-nadine.html/
https://www.papillesetpupilles.fr/2020/02/

https://www.papillesetpupilles.fr/2019/08/pates-a-lail-noir-au-sesame-et-au-basilic.html/
https://www.papillesetpupilles.fr/2019/08/tartines-aperitives-au-jambon-sec-coppa-et-rosette.html/
https://www.papillesetpupilles.fr/2019/08/salade-de-pois-chiche-a-lail-et-aux-deux-citrons.html/
https://www.papillesetpupilles.fr/2019/08/faire-ses-conserves-maison.html/
https://www.papillesetpupilles.fr/2019/08/paleron-de-boeuf-marine-au-bbq.html/
https://www.papillesetpupilles.fr/2019/08/pissaladiere.html/
https://www.papillesetpupilles.fr/2019/08/cuisses-de-poulet-au-four-tomates-et-thym.html/
https://www.papillesetpupilles.fr/2019/08/la-bouillabaisse-de-monique.html/
https://www.papillesetpupilles.fr/2019/08/gratin-daubergines-tomates-et-poivrons-au-parmesan.html/
https://www.papillesetpupilles.fr/2019/08/banh-xeo-les-crepes-salees-vietnamiennes.html/
https://www.papillesetpupilles.fr/2019/08/pickles-de-legumes.html/
https://www.papillesetpupilles.fr/2019/08/la-recette-des-37-tomates-farcies.ht

https://www.papillesetpupilles.fr/2019/02/porc-au-caramel-vietnam.html/
https://www.papillesetpupilles.fr/2019/02/moules-le-b-a-ba.html/
https://www.papillesetpupilles.fr/2019/02/nems-vietnam.html/
https://www.papillesetpupilles.fr/2019/02/sauce-aigre-douce-vietnam.html/
https://www.papillesetpupilles.fr/2019/02/puree-de-chou-fleur.html/
https://www.papillesetpupilles.fr/2019/02/radis-noir-et-crabe-en-remoulade.html/
https://www.papillesetpupilles.fr/2019/02/poulet-a-la-moutarde.html/
https://www.papillesetpupilles.fr/2019/02/les-decoupes-de-poulet-cuisses-pilons-blancs-ailes-aiguillettes.html/
https://www.papillesetpupilles.fr/2019/02/puree-de-brocolis-et-pommes-de-terre.html/
https://www.papillesetpupilles.fr/2019/01/endives-braisees-au-miel-et-au-jus-dorange.html/
https://www.papillesetpupilles.fr/2019/01/poulet-a-lorange.html/
https://www.papillesetpupilles.fr/2019/01/lapin-aux-olives.html/
https://www.papillesetpupilles.fr/2019/01/puree-de-pois-casse-au-thym-et-piment-despelette.h

https://www.papillesetpupilles.fr/2018/08/les-boulettes-parfaites-et-leur-sauce-aux-tomates-cerises.html/
https://www.papillesetpupilles.fr/2018/08/aubergines-roties-tomates-et-pesto.html/
https://www.papillesetpupilles.fr/2018/08/poulet-aux-tomates-et-aux-champignons.html/
https://www.papillesetpupilles.fr/2018/08/salade-de-mangue-verte-epicee-kalawang.html/
https://www.papillesetpupilles.fr/2018/08/salade-caprese.html/
https://www.papillesetpupilles.fr/2018/08/brochettes-de-saumon-sauce-au-basilic.html/
https://www.papillesetpupilles.fr/2018/08/la-sauce-de-lete-au-basilic.html/
https://www.papillesetpupilles.fr/2018/08/aubergines-a-la-sauce-sesame.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouverte-de-la-vanille-bleue-a-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/chouchous-christophines-au-jambon-et-bechamel.html/
https://www.papillesetpupilles.fr/2020/02/la-route-des-laves-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouverte-de-lhuile

https://www.papillesetpupilles.fr/2018/02/poulet-roti-aux-deux-pommes-de-terre-et-epices-douces.html/
https://www.papillesetpupilles.fr/2018/02/bricks-au-fromage-miel-poires-et-noix-de-pecan.html/
https://www.papillesetpupilles.fr/2018/01/comment-cuisiner-plat-mijote.html/
https://www.papillesetpupilles.fr/2018/01/salade-dendives-et-saumon-fume.html/
https://www.papillesetpupilles.fr/2018/01/joues-de-boeuf-braisees-au-vin-rouge.html/
https://www.papillesetpupilles.fr/2018/01/oeufs-en-meurette-faciles.html/
https://www.papillesetpupilles.fr/2018/01/gratin-de-poireaux-au-jambon.html/
https://www.papillesetpupilles.fr/2018/01/endives-braisees-a-lorange-et-au-pain-depices.html/
https://www.papillesetpupilles.fr/2018/01/aligot.html/
https://www.papillesetpupilles.fr/2018/01/coquelet-aux-pommes-de-terre-et-carottes.html/
https://www.papillesetpupilles.fr/2018/01/gratin-dendives-au-jambon.html/
https://www.papillesetpupilles.fr/2018/01/veloute-de-potimarron-et-chataignes.html/
https://www.pap

...

https://www.papillesetpupilles.fr/2008/03/thon-rouge-en-croute-de-sesame.html/
https://www.papillesetpupilles.fr/2008/03/brochettes-de-pruneaux-aux-rillettes.html/
https://www.papillesetpupilles.fr/2008/03/huitres-en-duo-terre-mer.html/
https://www.papillesetpupilles.fr/2008/03/risotto-aux-epinards-et-au-fromage-de.html/
https://www.papillesetpupilles.fr/2008/03/veloute-de-laitue.html/
https://www.papillesetpupilles.fr/2008/03/recette-de-risotto-de-jamie-oliver.html/
https://www.papillesetpupilles.fr/2008/02/recette-de-hamburger-l-orientale.html/
https://www.papillesetpupilles.fr/2008/02/recette-de-country-potatoes-pommes-de.html/
https://www.papillesetpupilles.fr/2008/02/recette-de-roti-de-thon-rouge-en-croute.html/
https://www.papillesetpupilles.fr/2008/01/recette-de-carottes-poelees-collantes.html/
https://www.papillesetpupilles.fr/2008/01/recette-de-lotte-grillee-de-jamie.html/
https://www.papillesetpupilles.fr/2008/01/recette-de-sauce-aux-olives-noires-de.html/
https://www.papille

https://www.papillesetpupilles.fr/2006/12/saint-jacques-la-poudre-de-rglisse.html/
https://www.papillesetpupilles.fr/2006/12/filets-de-bar-loup-sauce-vanille.html/
https://www.papillesetpupilles.fr/2006/12/sauce-vanille.html/
https://www.papillesetpupilles.fr/2006/11/oeufs-cocotte-au-foie-gras-et-au.html/
https://www.papillesetpupilles.fr/2006/11/butternut-en-gratin.html/
https://www.papillesetpupilles.fr/2006/11/filet-de-cabillaud-juste-rti.html/
https://www.papillesetpupilles.fr/2006/11/riz-santa-fe.html/
https://www.papillesetpupilles.fr/2006/11/crumble-la-polenta-et-la-ratatouille.html/
https://www.papillesetpupilles.fr/2006/11/quiche-thon-poireaux-parmesan.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouverte-de-la-vanille-bleue-a-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/chouchous-christophines-au-jambon-et-bechamel.html/
https://www.papillesetpupilles.fr/2020/02/la-route-des-laves-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouver

https://www.papillesetpupilles.fr/2006/04/papillotes-de-bar-loup-printanires.html/
https://www.papillesetpupilles.fr/2006/04/cuillers-de-la-mer-petit-amuse-bouche.html/
https://www.papillesetpupilles.fr/2006/04/garbure-barnaise-blog-apptit-7.html/
https://www.papillesetpupilles.fr/2006/04/moussaka-bulgare.html/
https://www.papillesetpupilles.fr/2006/04/epinards-aux-raisins-et-pignons-en.html/
https://www.papillesetpupilles.fr/2006/03/blanquette-de-lotte-au-safran.html/
https://www.papillesetpupilles.fr/2006/03/pure-de-pommes-de-terre-au-lait-de.html/
https://www.papillesetpupilles.fr/2006/03/papillotes-ensoleilles-de-cabillaud.html/
https://www.papillesetpupilles.fr/2006/03/coleslaw-faon-new-jersey.html/
https://www.papillesetpupilles.fr/2006/03/papillotes-de-bar-loup-au-fenouil.html/
https://www.papillesetpupilles.fr/2006/03/tartare-de-saumon-coleslaw-et-pignons.html/
https://www.papillesetpupilles.fr/2006/03/saut-de-veau-de-lisbonne.html/
https://www.papillesetpupilles.fr/2006/03/sau

https://www.papillesetpupilles.fr/2005/12/pole-de-champignons-de-paris.html/
https://www.papillesetpupilles.fr/2005/12/saint-jacques-panes-aux-noisettes.html/
https://www.papillesetpupilles.fr/2005/12/boudin-blanc-aux-pommes-en-corolles-de.html/
https://www.papillesetpupilles.fr/2005/12/lotte-aux-agrumes.html/
https://www.papillesetpupilles.fr/2005/12/pole-de-choux-de-bruxelles-chtaignes.html/
https://www.papillesetpupilles.fr/2005/12/carottes-braises.html/
https://www.papillesetpupilles.fr/2005/12/boeuf-bourguignon.html/
https://www.papillesetpupilles.fr/2005/12/ctes-de-veau-la-crme-destragon-et-au.html/
https://www.papillesetpupilles.fr/2005/11/epaule-dagneau-au-four.html/
https://www.papillesetpupilles.fr/2005/11/rmoulade-de-cleri-aux-poires.html/
https://www.papillesetpupilles.fr/2005/11/velout-aux-herbes.html/
https://www.papillesetpupilles.fr/2005/11/sluuurp-sluuurp-de-bleck-le-rock-ou.html/
https://www.papillesetpupilles.fr/2005/11/courge-spaghetti-en-sauce-tomate.html/
https://

https://www.papillesetpupilles.fr/2005/07/gratin-de-navets-aux-noisettes.html/
https://www.papillesetpupilles.fr/2005/07/lapin-aux-pruneaux.html/
https://www.papillesetpupilles.fr/2005/07/salade-de-poires-et-avocats-au-magret.html/
https://www.papillesetpupilles.fr/2005/07/fves-au-jambon.html/
https://www.papillesetpupilles.fr/2005/07/quiche-lorraine.html/
https://www.papillesetpupilles.fr/2005/07/salade-de-carmine-la-mangue.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouverte-de-la-vanille-bleue-a-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/chouchous-christophines-au-jambon-et-bechamel.html/
https://www.papillesetpupilles.fr/2020/02/la-route-des-laves-la-reunion.html/
https://www.papillesetpupilles.fr/2020/02/a-la-decouverte-de-lhuile-de-noix-du-perigord-au-moulin-de-la-veyssiere.html/
https://www.papillesetpupilles.fr/2017/09/brookgreen-gardens.html/
https://www.papillesetpupilles.fr/2017/09/port-de-larros-gujan-mestras-%e2%9b%b5%f0%9f%8c%9ebon-dimanche-a-to

In [784]:
#articles = [item for sublist in recipes for item in sublist]
#recipe_data = pd.DataFrame(recipes)

recipe_data.to_csv (r'/Users/vincentsalamand/Documents/Clubmama/Recipes/recipes_papillesetpupilles.csv', index = None, header=True)



In [3]:
recipe_data[recipe_data.name == "Lentilles à la Réunionnaise"]

NameError: name 'recipe_data' is not defined

In [400]:
# list recipe schema keys

recipe_keys = ["@context", "@type", "author", "cookTime", "prepTime", "totalTime", "performTime", "datePublished", 
               "name", "description", "image", "recipeIngredient", "nutrition", "recipeInstructions", "recipeYield", 
               "suitableForDiet", "recipeCategory", "publisher", "url", "keywords", "recipeCuisine"]

selectors = ["div", "span", "h1", "h2", "h3", "p", "li", "img", "ul", "ol", "meta", "time"]

meta_tags = ["og:site_name", "og:url"]

In [392]:
amandine_blog = "https://amandebasilic.com/recettes/"
driver.get(amandine_blog)
sleep(2)
soup_index = BeautifulSoup(driver.page_source, 'lxml')
index = soup_index.find('div', class_="index-layout").find('ul',  class_="index-list").find_all('li', classname="border-dotted")
recipe_urls = []
recipe_urls.append([link.find('a')['href'] for link in index])



In [405]:
soup_list = []

for recipe_url in recipe_urls[0]:
    page = "https://amandebasilic.com/" + recipe_url
    driver.get(page)
    sleep(1)
    print(page)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    soup_list.append(soup)

https://amandebasilic.com/chai-massala-bhoutan//
https://amandebasilic.com/chocolat-chaud-noel-feve-tonka//
https://amandebasilic.com/citronnade-menthe-melisse//
https://amandebasilic.com/creme-de-mure//
https://amandebasilic.com/detox-water-lilas-mures-citron//
https://amandebasilic.com/ginger-beer-biere-de-gingembre//
https://amandebasilic.com/kefir-de-fruits-figues-fraiches-et-citron//
https://amandebasilic.com/limonade-fleurs-de-sureau//
https://amandebasilic.com/liqueur-de-bourgeons-de-pin//
https://amandebasilic.com/liqueur-de-clementines//
https://amandebasilic.com/sirop-de-fleurs-de-mimosa-sauvage-cedrat//
https://amandebasilic.com/smoothie-detox-spiruline//
https://amandebasilic.com/smoothie-detox-arbouses-ananas-pomme//
https://amandebasilic.com/smoothie-melon-basilic-pour-lete//
https://amandebasilic.com/the-hibiscus//
https://amandebasilic.com/antipasti-courgettes-conserve/
https://amandebasilic.com/artichauts-a-la-romaine/
https://amandebasilic.com/betteraves-lactofermente

https://amandebasilic.com/tofu-maison/
https://amandebasilic.com/barre-de-cereales-avoine-et-coco-vegan/
https://amandebasilic.com/brioche-tang-zhong-fleur-doranger-vegan/
https://amandebasilic.com/chia-pudding-tarte-citron/
https://amandebasilic.com/focaccia-olives/
https://amandebasilic.com/fougasse-herbes-fraiches-tomates/
https://amandebasilic.com/galette-libanaise-zaatar-manouche/
https://amandebasilic.com/krantz-cake-vegan-yotam-ottolenghi/
https://amandebasilic.com/manala-saint-nicolas-vegan/
https://amandebasilic.com/muesli-croustillant-spiruline/
https://amandebasilic.com/pain-brioche-patate-douce-vegan/
https://amandebasilic.com/pain-de-mais-vegetalien/
https://amandebasilic.com/pain-maison-rose-betterave/
https://amandebasilic.com/pain-sans-gluten-sarrasin/
https://amandebasilic.com/panisse-ligurie-origan/
https://amandebasilic.com/papadums/
https://amandebasilic.com/petits-pains-au-raisins/
https://amandebasilic.com/porridge-healthy-epices-fruits-vegan/
https://amandebasili

In [487]:
amandine_recipes = []

for soup in soup_list:
    recipe_list = []
    for meta_tag in meta_tags:
        my_list = []
        my_list.append(meta_tag)
        my_list.append(soup.find('meta', property=meta_tag)["content"])
        recipe_list.append(my_list)

    for key in recipe_keys:
        my_list = []
        my_list.append(key)        
        for selector in selectors:
            try:
                content = []
                [content.append(item.text) for item in soup.find_all(selector, itemprop=key) if key is not "image"]
                [content.append(item["src"]) for item in soup.find_all(selector, itemprop=key) if key is "image"]
                if not content:
                    pass
                else:
                    my_list.append(content)
            except:
                pass
        recipe_list.append(my_list)

    recipe_dict = {}
    for list in recipe_list:
        values = list[1:]
        if len(values) > 0:
            recipe_dict[list[0]] = values[0]

    amandine_recipes.append(recipe_dict)

len(amandine_recipes)
        

238

In [490]:
pd.DataFrame(amandine_recipes).to_csv (r'/Users/vincentsalamand/Documents/Clubmama/Recipes/recipes_amandinebasilic.csv', index = None, header=True)
